# `ProteinWorkshop` Tutorial, Part 1 - Training a New Model
![Models](../docs/source/_static/box_models.png)

## Train a new model using the `ProteinWorkshop`

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

Welcome to the tutorial series for the `ProteinWorkshop`! 

In the `ProteinWorkshop`, we implement numerous [featurisation](https://www.proteins.sh/configs/features) schemes, [datasets](https://www.proteins.sh/configs/dataset) for [self-supervised pre-training](https://proteins.sh/quickstart_component/pretrain.html) and [downstream evaluation](https://proteins.sh/quickstart_component/downstream.html), [pre-training](https://proteins.sh/configs/task) tasks, and [auxiliary tasks](https://proteins.sh/configs/task.html#auxiliary-tasks).

[Processed datasets](https://drive.google.com/drive/folders/18i8rLST6ZICTBu6Q67ClT0KqN9AHeqoW?usp=sharing) and [pre-trained weights](https://drive.google.com/drive/folders/1zK1r8FpmGaqV_QwUJuvDacwSL0RW-Vw9?usp=sharing) are made available. Downloading datasets is not required; upon first run all datasets will be downloaded and processed from their respective source.

The `ProteinWorkshop` encompasses several models as well as pre-trained weights for them so that you can readily use them.

In this tutorial, we show you how you can use what is already available in the protein workshop to train and use models for specific tasks. The `ProteinWorkshop` is structured as a very modular package; we will therefore talk about how to change the different parts of it, like the model, training task, dataset, featurization scheme, etc. in this tutorial. 

Besides using all the different options we provide, you can make use of the modular nature of the `ProteinWorkshop` to add your own models, datasets, featurization schemes, and training tasks. We will show you how to do this in the next tutorials.

To train a new model, you can follow the following 3-step procedure:

1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks
2. Validate the designed training config
3. Use the designed config to train a new model

### 1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks

You can switch out any of these for another available option by replacing the corresponding argument's value in `overrides`:

`cfg = hydra.compose("template", overrides=["encoder=schnet", "task=inverse_folding", "dataset=afdb_swissprot_v4", "features=ca_base", "+aux_task=none"], return_hydra_config=True)`

In [2]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")
# print(rel_path)
GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=pronet",
        "encoder.level='aminoacid'",
        "encoder.num_blocks=4",
        "encoder.hidden_channels=128",
        # "encoder.out_channels=384",
        "encoder.mid_emb=64",
        "encoder.num_radial=6",
        "encoder.num_spherical=2",
        "encoder.cutoff=10.0",
        "encoder.max_num_neighbors=32",
        "encoder.int_emb_layers=3",
        "encoder.out_layers=2",
        "encoder.num_pos_emb=16",
        "encoder.dropout=0.3",
        "encoder.data_augment_eachlayer=True",
        "encoder.euler_noise=False",
        "encoder.pretraining=True",  #333333
        "encoder.node_embedding=False",

        # "decoder.graph_label.dummy=True",
        "decoder=subgraph_distances", # here
        "decoder.subgraph_distances.hidden_channels=128", # here

        "task=subgraph_distance_prediction", # here
        "dataset=afdb_swissprot_v4",
        "dataset.datamodule.batch_size=32",
        "features=fe_subgraph",  # here
        "+aux_task=none",
        
        "trainer.max_epochs=1000",
        "optimiser=adam",
        "optimiser.optimizer.lr=5e-4",
        "callbacks.early_stopping.patience=20", # to change, don't comment this
        "test=True",
        "scheduler=steplr", # to set parameters in yaml
        # "optimizer.weight_decay=0.5"
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[07/16/24 17:15:33] DEBUG    No `.env` file found in project root. Checking for env vars...         ]8;id=859765;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=389548;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#22\22]8;;\

                    DEBUG    No env var `DATA_PATH` found. Setting default...                       ]8;id=390434;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=103075;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#28\28]8;;\

                    INFO     DATA_PATH:                                                             ]8;id=276256;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=334455;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#39\39]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/data                          

[07/16/24 17:15:35] INFO     Hydra initialised at                                                    ]8;id=923694;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=317889;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/config.                       

### 2. Validate the designed training config

This is not strictly necessary, but it is a good idea to validate the config before training. This will check that all the arguments you have provided are valid and that the config is complete.

In [3]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

                    DEBUG    CUDA available: True                                                     ]8;id=80000;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=471564;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=357149;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=553113;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=624676;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=92334;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=795958;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=643846;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

In [4]:
print(cfg.keys())
for key in cfg.keys():
    print(key)
    print(cfg[key])

dict_keys(['hydra', 'env', 'dataset', 'features', 'encoder', 'decoder', 'transforms', 'callbacks', 'optimiser', 'scheduler', 'trainer', 'extras', 'metrics', 'task', 'logger', 'name', 'seed', 'num_workers', 'task_name', 'test', 'aux_task'])
hydra
{'run': {'dir': '${env.paths.log_dir}/${task_name}/runs/${now:%Y-%m-%d}_${now:%H-%M-%S}'}, 'sweep': {'dir': '${env.paths.log_dir}/${task_name}/multiruns/${now:%Y-%m-%d}_${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from those groups (group=option)\n\n$

### 3. Use the designed config to train a new model

Now with the config you have designed, you can train a new model. You can also use the `ProteinWorkshop` to evaluate the model on a downstream task.

In training:

ProteinBatch(fill_value=[32], atom_list=[32], coords=[15602, 37, 3], residues=[32], residue_id=[32], 
chains=[15602], residue_type=[15602], b_factor=[15602], id=[32], x=[15602, 23], seq_pos=[15602, 1], batch=[15602], 
ptr=[33], pos=[15602, 3], edge_index=[2, 247543], subgraphs=[1548, 149], subgraph_distances=[1548], 
subgraph_lengths=[1548])


In [5]:
from proteinworkshop.train import train_model

train_model(cfg)

Seed set to 52


[07/16/24 17:15:36] INFO     Instantiating datamodule:                                                  ]8;id=896664;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=987468;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#99\99]8;;\
                             <graphein.ml.datasets.foldcomp_dataset.FoldCompLightningDataModule...                 

100%|██████████| 542378/542378 [00:00<00:00, 4739598.39it/s]
Processing...
Done!


[07/16/24 17:15:37] INFO     Instantiating callbacks...                                                ]8;id=990167;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=446377;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#106\106]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.ModelCheckpoint>   ]8;id=440939;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=690437;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

...

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:368: UserWarning:

You have overridden `on_after_batch_transfer` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.



┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃     ┃ Name                                           ┃ Type        ┃ Params ┃ Mode  ┃    In sizes ┃   Out sizes ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 0   │ encoder                                        │ ProNet      │  1.4 M │ train │           ? │           ? │
│ 1   │ encoder.feature0                               │ d_theta_ph… │      0 │ train │  [[197606], │    [197606, │
│     │                                                │             │        │       │   [197606], │         24] │
│     │                                                │             │        │       │   [197606]] │             │
│ 2   │ encoder.feature1                               │ d_angle_emb │      0 │ train │  [[197606], │    [197606, │
│     │                                                │             │        │       │   [197606]] │         12] │
│ 3   │ encoder.embedding                              │ Embedding   │  3.3 K │ train │     [11080] │     [11080, │
│     │                                                │             │        │       │             │        128] │
│ 4   │ encoder.interaction_blocks                     │ ModuleList  │  1.4 M │ train │           ? │           ? │
│ 5   │ encoder.interaction_blocks.0                   │ Interactio… │  340 K │ train │    [[11080, │     [11080, │
│     │                                                │             │        │       │       128], │        128] │
│     │                                                │             │        │       │    [197606, │             │
│     │                                                │             │        │       │        24], │             │
│     │                                                │             │        │       │    [197606, │             │
│     │                                                │             │        │       │        12], │             │
│     │                                                │             │        │       │    [197606, │             │
│     │                                                │             │        │       │    16], [2, │             │
│     │                                                │             │        │       │    197606], │             │
│     │                                                │             │        │       │    [11080]] │             │
│ 6   │ encoder.interaction_blocks.0.dropout           │ Dropout     │      0 │ train │     [11080, │     [11080, │
│     │                                                │             │        │       │        128] │        128] │
│ 7   │ encoder.interaction_blocks.0.conv0             │ EdgeGraphC… │ 32.9 K │ train │    [[11080, │     [11080, │
│     │                                                │             │        │       │   128], [2, │        128] │
│     │                                                │             │        │       │    197606], │             │
│     │                                                │             │        │       │    [197606, │             │
│     │                                                │             │        │       │       128]] │             │
│ 8   │ encoder.interaction_blocks.0.conv0.aggr_module │ SumAggrega… │      0 │ train │   [[197606, │     [11080, │
│     │                                                │             │        │       │       128], │        128] │
│     │                                                │             │        │       │   [197606], │             │
│     │                                                │             │        │       │   '?', '?', │             │
│     │                                                │             │        │       │        '?'] │             │
│ 9   │ encoder.interaction_blocks.0.conv0.lin_l       │

Trainable params: 1.4 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.4 M                                                                                                
Total estimated model params size (MB): 5

Output()

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: UserWarning:

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any 
miscalculations, use `self.log(..., batch_size=batch_size)`.

### 4. Wrapping up

Have any additional questions about using the components provided in the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to leverage the full power of the repository!

/home/yang/anaconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py